<a href="https://colab.research.google.com/github/Radhikaagal/projectfacerecognition/blob/master/test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
 

Saving dataset.zip to dataset.zip
User uploaded file "dataset.zip" with length 16800298 bytes


In [0]:
import zipfile
zip_ref= zipfile.ZipFile('dataset.zip','r')
zip_ref.extractall()
zip_ref.close()

In [0]:
import patoolib
patoolib.extract_archive("facedataset.tar.gz", outdir=".")


patool: Extracting facedataset.tar.gz ...
patool: running /bin/tar --extract -z --file facedataset.tar.gz --directory .
patool: ... facedataset.tar.gz extracted to `.'.


'.'

In [0]:
import os
import pickle
import cv2
import matplotlib.pyplot as plt
import numpy as np

def create_data():
  x=[]
  y=[]
  label=0
  
  Base_dir = os.path.dirname(os.path.abspath('__file__'))
  data_dir = os.path.join(Base_dir,"dataset")
  
  for img_folder in os.listdir(data_dir):
      image_file=os.path.join(data_dir,img_folder)
      for file in os.listdir(image_file):
          img_path=os.path.join(image_file,file)
          img=cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)/256
          img=cv2.resize(img,(32,32))
          img=img.reshape(32,32,1)
          x.append(img)
          y.append(label)
      label=label+1
  return x,y
          
x,y=create_data()
np.save("xtrain.npy",x)
np.save("ytrain.npy",y)

In [0]:
import numpy as np
x=np.load("xtrain.npy")
y=np.load("ytrain.npy")
t_g = np.array([x[np.where(y==i)[0]] for i in np.unique(y)])# split the images of diffrent person
print(t_g.shape)
train = []
# Function to check similarity
def check_similarity(v):
    el=[]
    el.append(x[v[0]])
    el.append(x[v[1]])
    if y[v[0]]!=y[v[1]]:
      el.append(0)
    else:
      el.append(1)
    train.append(el)
    
# Function to generate only similar pair
def generate_similar(t_g):
   for i in range(0,50):
      for j in range(800):
        v=np.random.choice(len(t_g[i]),2)
        if v[0]!=v[1]:
          el2=[]
          el2.append(t_g[i][v[0]])
          el2.append(t_g[i][v[1]])
          el2.append(1)
          train.append(el2)

# Function to generate pair of images with labels
def generate_pair():
  for i in range(1,50):
    for j in range(1200): 
      v=np.random.choice(15*i,2)
      if v[0]!=v[1]:
        check_similarity(v)   
  generate_similar(t_g)
generate_pair()
t=np.array(train)
t.shape
np.save("pair_data.npy",t)
m=np.where(t[:,-1]==1)
print(len(t),len(m[0]))

    


(50, 15, 32, 32, 1)
95805 42501


In [0]:
from random import shuffle
import numpy as np
Data=np.load("pair_data.npy")
shuffle(Data)
x1=[]
x2=[]
y=[]

for i in range(95805):
  x1.append(Data[i][0])
  x2.append(Data[i][1])
  y.append(Data[i][2])

x1=np.array(x1)
x2=np.array(x2)
y=np.array(y)

np.save("x1.npy",x1)
np.save("x2.npy",x2)
np.save("y.npy",y)

X_train_data1=x1[:-38280]
X_train_data2=x2[:-38280]
Y_train_data=y[:-38280]

X_test_data1=x1[-38280:-19140]
X_test_data2=x2[-38280:-19140]
Y_test_data=y[-38280:-19140]

X_val_data1=x1[-19140:]
X_val_data2=x2[-19140:]
Y_val_data=y[-19140:]
print(X_train_data1.shape)


#Y_data=Data[:,-1]
#X_data=Data[:,:-1]
#X_train_data=X_data[:-38280]
#Y_train_data=Y_data[:-38280]

#X_test_data=X_data[-38280:-19140]
#X_test_data=Y_data[-38280:-19140]

#X_val_data=X_data[-19140:]
#Y_val_data=Y_data[-19140:]



(57525, 32, 32, 1)


In [0]:
import numpy as np
import tensorflow as tf
import keras
from keras.engine.input_layer import Input
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Lambda
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy

def compute_accuracy(predictions, labels):
    return labels[predictions.ravel() < 0.5].mean()

def create_siamese_model(input_shape):
  first_input = Input(input_shape)
  second_input = Input(input_shape)
  
  model = Sequential()
  model.add(Conv2D(32,(5,5),activation = 'relu', input_shape= input_shape))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.25))
  
  model.add(Conv2D(64,(3,3),activation = 'relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.5))
  
  model.add(Conv2D(128,(2,2),activation = 'relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.25))
  
  model.add(Flatten())
  
  model.add(Dense(128, activation='sigmoid'))
  
  encoded_first = model(first_input)
  encoded_second = model(second_input)
  
  L1_layer= Lambda(lambda f:K.abs(f[0]-f[1]))
  L1_distance = L1_layer([encoded_first,encoded_second])
  
  pred = Dense(1,activation = 'sigmoid')(L1_distance)
  
  siamese_network = Model(input=[first_input, second_input], outputs=pred)
  
  return siamese_network

mod= create_siamese_model((32,32,1))
# print(X_train_data[:,0].shape)
# mod.summary() 
batch_size = 5000
epoch = 300
optimizer = Adam(0.0006)
mod.compile(loss="binary_crossentropy",optimizer=optimizer, metrics=['accuracy'])
history = mod.fit([X_train_data1,X_train_data2], Y_train_data, batch_size=batch_size, epochs=epoch,verbose=1, validation_data = ([X_val_data1,X_val_data2], Y_val_data))
pred = mod.predict([X_train_data1,X_train_data2])
tr_acc = compute_accuracy(pred, Y_train_data)
pred = mod.predict([X_test_data1, X_test_data2])
te_acc = compute_accuracy(pred, Y_test_data)
print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=[<tf.Tenso...)`


Train on 57525 samples, validate on 19140 samples
Epoch 1/10
57525/57525 [==============================] - 230s 4ms/step - loss: 0.6945 - acc: 0.4803 - val_loss: 0.6893 - val_acc: 0.6255
Epoch 2/10
57525/57525 [==============================] - 225s 4ms/step - loss: 0.6642 - acc: 0.7684 - val_loss: 0.6421 - val_acc: 0.6255
Epoch 3/10
57525/57525 [==============================] - 225s 4ms/step - loss: 0.5933 - acc: 0.7780 - val_loss: 0.6029 - val_acc: 0.6255
Epoch 4/10
57525/57525 [==============================] - 226s 4ms/step - loss: 0.4898 - acc: 0.7821 - val_loss: 0.5622 - val_acc: 0.6255
Epoch 5/10
57525/57525 [==============================] - 225s 4ms/step - loss: 0.4004 - acc: 0.7951 - val_loss: 0.5738 - val_acc: 0.6255
Epoch 6/10
57525/57525 [==============================] - 226s 4ms/step - loss: 0.3637 - acc: 0.8195 - val_loss: 0.5954 - val_acc: 0.6255
Epoch 7/10
57525/57525 [==============================] - 225s 4ms/step - loss: 0.3504 - acc: 0.8339 - val_loss: 0.6106 - 